In [26]:
from ctypes import *
import time
from sys import path
import sys
from os import sep
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

if sys.platform.startswith("win"):
    dwf = cdll.dwf
    constants_path = "C:" + sep + "Program Files (x86)" + sep + "Digilent" + sep + "WaveFormsSDK" + sep + "samples" + sep + "py"
elif sys.platform.startswith("darwin"):
    dwf = cdll.LoadLibrary("/Library/Frameworks/dwf.framework/dwf")
    constants_path = "/Applications/WaveForms.app/Contents/Resources/SDK/samples/py"
else:
    dwf = cdll.LoadLibrary("libdwf.so")
    constants_path = "/usr/share/digilent/waveforms/samples/py"

path.append(constants_path)
import dwfconstants as constants

In [27]:
dwf.FDwfDeviceCloseAll()
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value)
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))

hdwf = c_int()
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
if hdwf.value == 0:
    raise RuntimeError("Failed to open device.")
print(hdwf.value)

dwf.FDwfAnalogIOReset(hdwf)
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(3.0))
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1))
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(1))
time.sleep(0.5)

vpos = c_double()
dwf.FDwfAnalogIOStatus(hdwf)
dwf.FDwfAnalogIOChannelNodeStatus(hdwf, c_int(0), c_int(1), byref(vpos))
print(f"Power: +{vpos.value:.2f}V")

1
Power: +3.00V


In [28]:
DEVICE_ADDR = 0x52
ENABLE_REG = 0x80
ATIME_REG = 0x81
CONTROL_REG = 0x8F
CDATAL_REG = 0x94
CDATAH_REG = 0x95
RDATAL_REG = 0x96
RDATAH_REG = 0x97
GDATAL_REG = 0x98
GDATAH_REG = 0x99
BDATAL_REG = 0x9A
BDATAH_REG = 0x9B

def write_register(reg, value):
    iNak = c_int()
    rgTX = (c_ubyte * 2)(reg, value)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(2), byref(iNak))
    return iNak.value == 0

def read_register(reg):
    iNak = c_int()
    rgTX = (c_ubyte * 1)(reg)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(1), byref(iNak))
    if iNak.value == 0:
        rgRX = (c_ubyte * 1)()
        dwf.FDwfDigitalI2cRead(hdwf, c_int(DEVICE_ADDR), rgRX, c_int(1), byref(iNak))
        if iNak.value == 0:
            return rgRX[0]
    return None

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3))
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(0))
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(1))
iNak = c_int()
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))

1

In [29]:
write_register(ENABLE_REG, 0x01)
time.sleep(0.01)
write_register(ENABLE_REG, 0x03)
write_register(ATIME_REG, 0xFF)
write_register(CONTROL_REG, 0x02)

def read_colors():
    time.sleep(0.1)
    
    clear_low = read_register(CDATAL_REG)
    clear_high = read_register(CDATAH_REG)
    red_low = read_register(RDATAL_REG)
    red_high = read_register(RDATAH_REG)
    green_low = read_register(GDATAL_REG)
    green_high = read_register(GDATAH_REG)
    blue_low = read_register(BDATAL_REG)
    blue_high = read_register(BDATAH_REG)
    
    if all(val is not None for val in [clear_low, clear_high, red_low, red_high, green_low, green_high, blue_low, blue_high]):
        clear = (clear_high << 8) | clear_low
        red = (red_high << 8) | red_low
        green = (green_high << 8) | green_low
        blue = (blue_high << 8) | blue_low
        return {'clear': clear, 'red': red, 'green': green, 'blue': blue}
    return None

In [32]:
data1={'int_time': [], 'clear': [], 'red': [], 'green': [], 'blue': []}

write_register(ATIME_REG, 0xFF) # Integration Time value FF
atime_arr = [0xFF, 0xF6, 0xD5, 0xC0, 0x00]

for atime_values in atime_arr:
    write_register(ATIME_REG, atime_values)
    samples = []
    for i in range(100):
        Int_time = (256 - atime_values) * 2.4
        wait_time = (Int_time/1000)+ 0.01
        time.sleep(wait_time)
        
        colors = read_colors()
        if colors: samples.append(colors['clear'])
        data1['int_time'].append(Int_time)
        data1['clear'].append(colors['clear'])
        data1['red'].append(colors['red'])
        data1['green'].append(colors['green'])
        data1['blue'].append(colors['blue'])
        if i%10==0:
            print(f"Integration Time: {Int_time:.2f}: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")
        
    samples = np.array(samples)
    samples_mean = np.mean(samples)
    samples_std = np.std(samples)
    samples_jitter = samples_std / samples_mean * 100
    print(f"Mean: {samples_mean:.2f}, Std Dev: {samples_std:.2f}, Jitter: {samples_jitter:.2f}%")    
    print("-----"*10)

Integration Time: 2.40: 67, R: 14, G: 24, B: 16
Integration Time: 2.40: 64, R: 14, G: 23, B: 15
Integration Time: 2.40: 64, R: 14, G: 18, B: 20
Integration Time: 2.40: 68, R: 22, G: 24, B: 29
Integration Time: 2.40: 87, R: 18, G: 31, B: 22
Integration Time: 2.40: 71, R: 22, G: 25, B: 29
Integration Time: 2.40: 85, R: 19, G: 30, B: 28
Integration Time: 2.40: 74, R: 18, G: 27, B: 23
Integration Time: 2.40: 64, R: 18, G: 23, B: 23
Integration Time: 2.40: 44, R: 15, G: 16, B: 17
Mean: 67.98, Std Dev: 12.83, Jitter: 18.87%
--------------------------------------------------
Integration Time: 24.00: 544, R: 151, G: 193, B: 170
Integration Time: 24.00: 532, R: 146, G: 189, B: 167
Integration Time: 24.00: 619, R: 169, G: 233, B: 212
Integration Time: 24.00: 886, R: 208, G: 315, B: 291
Integration Time: 24.00: 873, R: 206, G: 311, B: 295
Integration Time: 24.00: 854, R: 202, G: 305, B: 289
Integration Time: 24.00: 887, R: 208, G: 315, B: 298
Integration Time: 24.00: 847, R: 211, G: 318, B: 300
I

In [36]:
data1={'int_time': [], 'clear': [], 'red': [], 'green': [], 'blue': []}

write_register(ATIME_REG, 0xFF) # Integration Time value FF
gain_arr = {"x1": 0x00, "x4": 0x01, "x16": 0x02, "x64": 0x03}

for mult, gain_values in gain_arr.items():
    write_register(CONTROL_REG, gain_values)
    samples = []
    for i in range(100):
        Int_time = (256 - 0xFF) * 2.4
        wait_time = (Int_time/1000)+ 0.01
        time.sleep(wait_time)
        
        colors = read_colors()
        if colors: samples.append(colors['clear'])
        data1['int_time'].append(Int_time)
        data1['clear'].append(colors['clear'])
        data1['red'].append(colors['red'])
        data1['green'].append(colors['green'])
        data1['blue'].append(colors['blue'])
        if i%10==0:
            print(f"Gain: {mult}: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")
        
    samples = np.array(samples)
    samples_mean = np.mean(samples)
    samples_std = np.std(samples)
    samples_jitter = samples_std / samples_mean * 100
    print(f"Mean: {samples_mean:.2f}, Std Dev: {samples_std:.2f}, Jitter: {samples_jitter:.2f}%")    
    print("-----"*10)

Gain: x1: 6, R: 1, G: 2, B: 1
Gain: x1: 4, R: 2, G: 1, B: 2
Gain: x1: 5, R: 1, G: 2, B: 2
Gain: x1: 4, R: 1, G: 1, B: 2
Gain: x1: 6, R: 1, G: 2, B: 1
Gain: x1: 6, R: 1, G: 1, B: 2
Gain: x1: 5, R: 1, G: 2, B: 1
Gain: x1: 5, R: 1, G: 2, B: 1
Gain: x1: 3, R: 0, G: 1, B: 1
Gain: x1: 4, R: 2, G: 1, B: 1
Mean: 4.68, Std Dev: 1.20, Jitter: 25.62%
--------------------------------------------------
Gain: x4: 16, R: 6, G: 6, B: 8
Gain: x4: 19, R: 6, G: 7, B: 7
Gain: x4: 23, R: 5, G: 8, B: 7
Gain: x4: 18, R: 7, G: 6, B: 8
Gain: x4: 18, R: 7, G: 6, B: 9
Gain: x4: 18, R: 7, G: 6, B: 9
Gain: x4: 21, R: 6, G: 7, B: 9
Gain: x4: 24, R: 5, G: 9, B: 8
Gain: x4: 20, R: 6, G: 7, B: 8
Gain: x4: 20, R: 6, G: 7, B: 9
Mean: 22.31, Std Dev: 3.32, Jitter: 14.90%
--------------------------------------------------
Gain: x16: 76, R: 28, G: 27, B: 38
Gain: x16: 112, R: 18, G: 39, B: 25
Gain: x16: 100, R: 18, G: 35, B: 23
Gain: x16: 98, R: 20, G: 35, B: 24
Gain: x16: 95, R: 20, G: 34, B: 26
Gain: x16: 81, R: 22, G: 2

In [37]:
def cleanup():
    dwf.FDwfDeviceClose(hdwf)
cleanup()